# Dependências

In [ ]:
#!pip install numba==0.49.1
!pip install numba==0.56.4
#!pip install numba
#!pip uninstall numba -y
#!pip install llvmlite

In [ ]:
#!pip install llvmlite --ignore-installed

In [ ]:
#!pip install gensim==3.6.0
#!pip install binary gensim==3.6.0

In [ ]:
#!pip3 uninstall numpy -y

In [ ]:
#!pip install numpy

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import psycopg2 as postgres
import psycopg2.extras
import math
import csv
import random

from gensim.models import Word2Vec
from tqdm import tqdm
#from sshtunnel import SSHTunnelForwarder
from tqdm import tqdm

import numba
from numba import njit
#from numba import cuda

# Funções

## Utilidades

In [2]:
#LENTO
def calculate_L(POI_path, df):
    dst = 0
    segment = []
    for poi_id in POI_path:
        #print(poi_id)
        segment.append(poi_id)
        if (len(segment) > 2):
            segment.pop(0)

        if (len(segment) == 2):

            p_idx = POI_path[-2]
            p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]
            n_idx = POI_path[-1]
            n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]
        
            dst += distances_matrix[p_idx][n_idx]
            
    return dst

In [3]:
#Meio LENTO
def calculate_L_(POI_path, df):
    dst = 0
    
    for i in range(len(POI_path)-1):

        p_idx = POI_path[i]
        p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]
        n_idx = POI_path[i+1]
        n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]

        dst += distances_matrix[p_idx][n_idx]
            
    return dst

In [4]:
#Mais Rápido
def calculate_Li_(POI_path, df, i, L):

    p_idx = POI_path[i-1]
    p_idx = df.query("business_id=='"+str(p_idx)+"'").index[0]

    c_idx = POI_path[i]
    c_idx = df.query("business_id=='"+str(c_idx)+"'").index[0]

    n_idx = POI_path[i+1]
    n_idx = df.query("business_id=='"+str(n_idx)+"'").index[0]

    #Retirando antigo segmento
    segment_distance = L - distances_matrix[p_idx][n_idx]
    
    #Tamanho do novo segmento
    dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]
            
    return dst

In [5]:
#Essa versão não utilizada a otimização do Numba
def generate_segment(W, L, POI_path):
    
    final_path = [pt for pt in POI_path]

    for wp in W:

        shortest_path = np.inf
        optimal_list = None

        #Testar WP em todas as posições entre 1 e len(list)-2
        for j in range(1, len(final_path)):
            
            local_list = [fp for fp in final_path]
            
            #Testar WP em todas as posições entre 1 e len(list)-2
            local_list.insert(j, wp)
            

            p_idx = j-1
            c_idx = j
            n_idx = j+1

            #Retirando antigo segmento
            segment_distance = L - distances_matrix[p_idx][n_idx]

            #Tamanho do novo segmento
            dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]

            if (dst < shortest_path):
                shortest_path = dst
                optimal_list = [ll for ll in local_list] 
            
        final_path = [ol for ol in optimal_list] 
        L = shortest_path
    
    return final_path

In [6]:
# function optimized to run on gpu 
#@jit(target_backend='cuda')
#@numba.njit
#from numba.typed import List

@njit
#@cuda.jit
def generate_segment_numba(W, L, POI_path):
    
    final_path = [pt for pt in POI_path]

    for wp in W:

        shortest_path = np.inf
        optimal_list = None

        #Testar WP em todas as posições entre 1 e len(list)-2
        for j in range(1, len(final_path)):
            
            local_list = [fp for fp in final_path]
            
            #Testar WP em todas as posições entre 1 e len(list)-2
            local_list.insert(j, wp)
            

            p_idx = j-1 
            c_idx = j
            n_idx = j+1

            #Retirando antigo segmento
            segment_distance = L - distances_matrix[p_idx][n_idx]

            #Tamanho do novo segmento
            dst = segment_distance + distances_matrix[p_idx][c_idx] + distances_matrix[c_idx][n_idx]

            if (dst < shortest_path):
                shortest_path = dst
                optimal_list = [ll for ll in local_list] 
            
        final_path = [ol for ol in optimal_list] 
        L = shortest_path
    
    return final_path

In [7]:
from timeit import default_timer as timer    

# normal function to run on cpu 
def func(a):                                 
    for i in range(100000): 
        a[i]+= 1      

# function optimized to run on gpu  
#@jit(nopython=True)
#@cuda.jit
#(target ="cuda")    
@njit
def func2(a): 
    for j in range(100000):
        for i in range(100000): 
            a[i]+= 1
#if __name__=="__main__": 

n = 100000                           
a = np.ones(n, dtype = np.float64) 
#b = np.ones(n, dtype = np.float32) 

start = timer() 
func(a) 
print("without GPU:", timer()-start)     

start = timer() 
func2(a) 
print("with GPU:", timer()-start) 

without GPU: 0.05708941211923957
with GPU: 4.9828040360007435


# Document Corpus

# Carregando dados de POIs

In [2]:
df_NYC

NameError: name 'df_NYC' is not defined

In [8]:
df_NYC = pd.read_csv("../dataset/NYC/NYC_train.csv")
df_NYC_ = df_NYC[["POI_id", "latitude", "longitude"]]
df_NYC_.head()

,POI_id,latitude,longitude
0,49bbd6c0f964a520f4531fe3,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,40.741191,-73.989663


In [17]:
#Teste
df = df_NYC_[:10000]
df.head()

,POI_id,latitude,longitude
0,49bbd6c0f964a520f4531fe3,40.719810,-74.002581
1,4a43c0aef964a520c6a61fe3,40.606800,-74.044170
2,4c5cc7b485a1e21e00d35711,40.716162,-73.883070
3,4ce1863bc4f6a35d8bd2db6c,40.619151,-74.035888
4,4c0ab56f7e3fc9288c1df482,40.741191,-73.989663


In [9]:
df_NYC_.shape

(83228, 3)

In [10]:
df_NYC_ = df_NYC_.drop_duplicates(subset=["POI_id"])
df_NYC_.shape

(4980, 3)

In [11]:
#Teste
from scipy.spatial.distance import cdist
distances_matrix = cdist(df_NYC_[['latitude', 'longitude']].values, df_NYC_[['latitude', 'longitude']].values)
distances_matrix[0]

array([0.        , 0.12042037, 0.11956666, ..., 0.21476466, 0.05929455,
       0.21344547])

In [13]:
distances_matrix[0]

array([0.        , 0.12042037, 0.11956666, ..., 0.21476466, 0.05929455,
       0.21344547])

In [12]:
mvps = df_NYC_["POI_id"].value_counts()
mvps

49bbd6c0f964a520f4531fe3    1
4dcc5861887714552bd47b34    1
4b42a69ff964a52043d825e3    1
4a11d75df964a52050771fe3    1
4c8509cfd4e2370470c67e88    1
                           ..
4a3966c7f964a520689f1fe3    1
4ba50a3ef964a5204cd438e3    1
4c1a47228b3aa5936051955f    1
4ec27c885c5c3d470deb0904    1
503bee84e4b0e79b22202bf1    1
Name: POI_id, Length: 4980, dtype: int64

In [13]:
"49bbd6c0f964a520f4531fe3" in df_NYC_["POI_id"].value_counts()

True

In [14]:
# TESTE LOCAL

#object_id é uma POI_catid
object_id = "4dcc5861887714552bd47b34"	
poi_df = df_NYC_

# print('processing POI:', poi_id)
POI_path_bi = []
POI_path_fl = []

# print('processing:', object_id)

# Obtendo informações de categorias e checkin do poi central
# [business_id, checkin, category]
# poi_information = getZoningInformation(connection, object_id)
poi_information = poi_df[['POI_id']]

# Executa o processo se há dados
# Um caminho preciso do no mínimo dois POIs
if len(poi_information) >= 2:

    # Restringindo as possibilidades de tipos
    columns = list(dict(poi_information.iloc[0]).keys())
    poi_information = pd.DataFrame(poi_information, columns=columns)

    # Shortest Path construction
    pois_idx = []
    for i, cp_idx in poi_information.iterrows():
        pois_idx.append(poi_df.query("POI_id=='" + str(cp_idx['POI_id']) + "'").index[0])

    # Encontrar Ps e Pe
    max_distances = []
    #for poi_idx_01 in pois_idx:
    #    for poi_idx_02 in pois_idx:
    #        if poi_idx_01 != poi_idx_02:
    #            max_distances.append((poi_idx_01, poi_idx_02, distances_matrix[poi_idx_01][poi_idx_02]))

    for i in range(len(pois_idx)):
        for j in range(len(pois_idx)):
            if pois_idx[i] != pois_idx[j]:
                max_distances.append((pois_idx[i], pois_idx[j], distances_matrix[i][j]))
                                    
    furthest_pair = max_distances[0]
    for md in max_distances:
        if md[2] > furthest_pair[2]:
            furthest_pair = md

    POI_path = [furthest_pair[0], furthest_pair[1]]
    L = furthest_pair[2]

    # A is all POIs
    W = set(pois_idx).difference(set(POI_path))

    # Ajuste das variáveis para a função generate_segment
    W = list(W)

    # Adicionar o POI de A que tem o menor tamanho de caminho em L e atualizar W
    if len(W) > 0:

        # Não otimizado
        POI_path_fl = generate_segment(W, L, POI_path)

        # Otimizado pelo numba
        #POI_path_fl = generate_segment_numba(W, L, POI_path)
    else:

        POI_path_fl = POI_path.copy()

    POI_path_bi = [poi_df.iloc[pp]['POI_id'] for pp in range(len(POI_path_fl))]

In [23]:
POI_path_fl

[3138,
 2,
 1,
 0,
 6,
 5,
 4,
 3,
 123,
 122,
 121,
 120,
 119,
 118,
 117,
 116,
 115,
 114,
 113,
 112,
 111,
 110,
 109,
 108,
 107,
 106,
 105,
 104,
 103,
 102,
 101,
 100,
 99,
 98,
 97,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 88,
 87,
 86,
 85,
 84,
 83,
 82,
 81,
 80,
 79,
 78,
 77,
 76,
 75,
 74,
 73,
 72,
 71,
 70,
 69,
 68,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 59,
 58,
 57,
 56,
 55,
 54,
 53,
 52,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 35,
 34,
 16411,
 32,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 12,
 11,
 10,
 9,
 8,
 7,
 81913,
 81903,
 16361,
 81886,
 32728,
 49093,
 81814,
 32654,
 49014,
 49000,
 32612,
 16221,
 81750,
 16213,
 16202,
 16170,
 16143,
 32519,
 32504,
 16103,
 81632,
 32476,
 81623,
 16077,
 16075,
 48837,
 48822,
 16053,
 32428,
 16036,
 48794,
 16026,
 16020,
 32398,
 16008,
 16002,
 15999,
 32382,
 15987,
 32366,
 15980,
 32357,
 48743,
 81509,
 159

In [ ]:
for pp in POI_path_fl:
    POI_path_bi2.append(poi_df.iloc[pp]['POI_id'])

In [27]:
#RESULTADO
POI_path_

['49bbd6c0f964a520f4531fe3',
 '4a43c0aef964a520c6a61fe3',
 '4c5cc7b485a1e21e00d35711',
 '4ce1863bc4f6a35d8bd2db6c',
 '4c0ab56f7e3fc9288c1df482',
 '4c00297392a6c9280cc243e2',
 '4b9830a5f964a520c73235e3',
 '4ba8a9a9f964a5206ee539e3',
 '4c955b3558d4b60c06d03129',
 '4ba0e0edf964a520138437e3',
 '4a676ff4f964a5206fc91fe3',
 '4c5ef77bfff99c74eda954d3',
 '4bcde547511f95210d62b5c7',
 '3fd66200f964a52035e41ee3',
 '4b3a8e5df964a520d86925e3',
 '4b55ec3bf964a520aaf627e3',
 '40b68100f964a5207d001fe3',
 '4bf5cedf4d5f20a1833d98fe',
 '40e74880f964a520150a1fe3',
 '4ad607d3f964a520a10421e3',
 '49b15042f964a520d4521fe3',
 '4ca66eec5a1e952129d98ace',
 '4c79b79381bca093e689fe14',
 '447bf8f1f964a520ec331fe3',
 '4dbdebed6e810768bf82ad95',
 '4e79392ce4cdb158f1cf4b75',
 '4bdb4bf54b1f952169ac670b',
 '4b093cd0f964a520db1423e3',
 '4c17d9974ff90f47a9890d49',
 '4ab11744f964a520046820e3',
 '4b298573f964a52050a024e3',
 '4c1d0f98c09ed13ad4c5828e',
 '4b7c9fe1f964a520559e2fe3',
 '4a7a0832f964a5203ce81fe3',
 '4e5dbf1e1f6e

In [26]:
POI_path_bi[0]

'49bbd6c0f964a520f4531fe3'

In [15]:
#TIPO DE DADO DO POI_INFORMATION
poi_information

,POI_id
0,49bbd6c0f964a520f4531fe3
1,4a43c0aef964a520c6a61fe3
2,4c5cc7b485a1e21e00d35711
3,4ce1863bc4f6a35d8bd2db6c
4,4c0ab56f7e3fc9288c1df482
...,...
82872,50988e54e4b0fd23874c1700
82886,505b3f57e4b0c1e23aa7cd5e
82890,4d07e5203593548174e0f529
82947,4fbd65f6e4b04d6c380fe8df


In [ ]:
#DATAFRAME DE POI
poi_df = poi_df[['POI_id', 'latitude', 'longitude']]
poi_df.head()

# Carregando dados de Zonas

In [26]:
obj_id_list = list(final_zoning_gdf['objectid'])
obj_id_list[:5]

NameError: name 'final_zoning_gdf' is not defined

In [ ]:
results = []
for obj_id in tqdm(obj_id_list):
    results.append(Shortest_Path_FT(obj_id, poi_df))

In [25]:
Shortest_Path_FT("4dcc5861887714552bd47b34", df_NYC_)

NameError: name 'Shortest_Path_FT' is not defined

In [21]:
#file_name = './Atlanta/Shortest Path/atlanta-shortest_path-zoning-FT_gpu.csv'
file_name = './NYC-shortest_path-Copy3.csv'


#with open(file_name, mode="w", newline="") as arquivo:
#    writer = csv.writer(arquivo)
#    writer.writerow(["ID"])
#    for item in POI_path_bi:
#        writer.writerow([item])

#print(f"Arquivo '{file_name}' criado com sucesso!")


csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in POI_path_bi:
    if(len(result) > 0):
        writer_path.writerow(result)
print(f"Arquivo '{file_name}' criado com sucesso!")

Arquivo './NYC-shortest_path-Copy3.csv' criado com sucesso!


## Execução em Paralelo

In [ ]:
import multiprocessing as mp

from timeit import default_timer as timer
start = timer() 

print("Number of processors: ", mp.cpu_count())

# Step 1: Init multiprocessing.Pool()
#pool = mp.Pool(int(mp.cpu_count()))
pool = mp.Pool(int(2))

# Step 2: `pool.apply` the `howmany_within_range()`
items = [(object_id, poi_df) for object_id in obj_id_list]

results = pool.starmap(Shortest_Path_FT, items)

#Step 2.1 Writing results

#file_name = './geographic/SHORTEST PATH/austin-shortest_path-zoning-FT_gpu.csv'
file_name = './Atlanta/Shortest Path/atlanta-shortest_path-zoning-FT_gpu.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    writer_path.writerow(result)

csv_file_02.close()

# Step 3: Don't forget to close
pool.close()
    
#print("without GPU:", timer()-start)
print("with GPU:", timer()-start) 

In [ ]:
print(len(results))

In [ ]:
#file_name = './geographic/SHORTEST PATH/austin-sl-corpus-' + str(bin_number) + '-c_.csv'
file_name = './geographic/SHORTEST PATH/austin-shortest_path-' + str(bin_number) + '-CF-c_.csv'
#file_name = './geographic/SHORTEST PATH/austin-shortest_path-' + str(bin_number) + '-FT-c.csv'

csv_file_02 = open(file_name, "w", newline='')
writer_path = csv.writer(csv_file_02, delimiter=',')
for result in results:
    writer_path.writerow(result)

In [ ]:
print('ok')